***
# Fundamentals of Data Analytics Assessment - CAO Points Analysis

***

[2021 CAO points]('http://www2.cao.ie/points/l8.php')

[Regular Expression Documentation]('https://docs.python.org/3/library/re.html')

#### Importing Libraries 

In [1]:
# Regular expressions package
import re 

# Package for HTTP requests.
import requests as rq

# Date and time 
import datetime as dt

#### Server Request

In [25]:
# Getting CAO points URL.
response = rq.get('http://www2.cao.ie/points/l8.php')

# 200 = ok. 404 = error: not found
response

<Response [200]>

### Save Orignal Dataset

***

In [26]:
# Getting current dat and time
now = dt.datetime.now()

# Format as a string (year-month-day hour:minute:second)
string_now = now.strftime('%Y%m%d_%H%M%S')

In [27]:
# File path to orginal data
path = 'cao_data/cao2021_' + string_now + '.html'

### Webserver Error

***

Webserver error - server says decode as:

    Content-Type: text/html; charset=iso-8859-1
However, one line uses \x96 which isn't defined in iso-8859-1. 

Therefore, a similar decoding standard cp1252 was used. It is similar but inculdes \x96

In [28]:
# Fixing the webserves wrong encoding
orignal_encoding = response.encoding

# Changing to cp1252
response.encoding = 'cp1252'

In [29]:
# Saving the orginal html file.
with open(path, 'w') as f: # Opening path in write mode
    f.write(response.text)

FileNotFoundError: [Errno 2] No such file or directory: 'cao_data/cao2021_20211018_203655.html'

### Getting relevant data using Regular expressions

***

To get the relevant lines from the response request, we use a regular expression. It is more efficent that recalling the expression everytime.

In [30]:
# Complie regualr expression for any matching lines. 
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)([0-9]{3})(\*?) *') #r for raw data

Loop through the response for matches 

In [31]:
# Path to csv file
path = 'cao_data\cao2021_csv_' +  string_now + '.csv'

In [32]:
# Keeping track of the number of courses
no_lines = 0

path = 'cao_data\cao2021_csv_' + string_now + '.csv'

# Keep track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path, 'w') as f:
# Open the csv file for writing.
#with open(path, 'w') as f:
    # Loop through lines of the response.
    for line in response.iter_lines():
            # Decode the line, using the wrong encoding!
            dline = line.decode('cp1252')
            # Match only the lines representing courses.
            if re_course.fullmatch(dline):
                # Add one to the lines counter.
                no_lines = no_lines + 1
                csv_version = re_course.sub(r'\1,\2,\3,\4', dline) # Getting relevant parts for lines which match
                #print(csv_version) # Print CSV style
                f.write(csv_version + '/n')

        # Number of courses matched
print(f'Total number of lines matched: {no_lines}.')
    

FileNotFoundError: [Errno 2] No such file or directory: 'cao_data\\cao2021_csv_20211018_203655.csv'

## References

## End